<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/shieh2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

paper : https://doi.org/10.1371/journal.pone.0233188



 " IDH and MGMT markers were not mandatory for routine pathologic reports " Sadly we won't have this markers in this one, but we have it in other datasets.

In [16]:
dataset = pd.read_excel('/content/shieh.xlsx')

In [17]:
dataset.head(5)

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1,63,1,1,1,6000,1.0,1,1,2,1.36,1.260274,731.0,1,63
1,1,64,1,1,0,6600,0.0,2,1,1,8.84,5.115068,NaN,1,64
2,1,62,1,1,1,6000,1.0,1,1,2,1.46,0.641096,215.0,1,62
3,1,71,1,1,1,6000,NaN,3,1,1,2.42,0.652055,255.0,1,71
4,1,21,1,1,0,5940,0.0,2,1,2,1.26,1.079452,154.0,1,21


In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Let's do all the pre-processing needed

In [18]:
imp_mean = IterativeImputer(random_state=0)

In [19]:
data = imp_mean.fit_transform(dataset)

In [20]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [21]:
data.head()

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1.0,63.0,1.0,1.0,1.0,6000.0,1.0,1.0,1.0,2.0,1.36,1.260274,731.000000,1.0,63.0
1,1.0,64.0,1.0,1.0,0.0,6600.0,0.0,2.0,1.0,1.0,8.84,5.115068,272.938289,1.0,64.0
2,1.0,62.0,1.0,1.0,1.0,6000.0,1.0,1.0,1.0,2.0,1.46,0.641096,215.000000,1.0,62.0
3,1.0,71.0,1.0,1.0,1.0,6000.0,-1.0,3.0,1.0,1.0,2.42,0.652055,255.000000,1.0,71.0
4,1.0,21.0,1.0,1.0,0.0,5940.0,0.0,2.0,1.0,2.0,1.26,1.079452,154.000000,1.0,21.0


In [22]:
data.corr(method = 'pearson')

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
gender,1.000000,-0.163487,-0.139923,-0.113228,0.169842,0.117790,0.159428,-0.153072,-0.030024,-0.037493,0.043419,0.080638,-0.073217,1.000000,-0.163487
age,-0.163487,1.000000,0.202956,0.075994,-0.005687,-0.199397,0.046168,0.037592,0.110030,0.089632,-0.188313,-0.254452,0.153293,-0.163487,1.000000
statusPFS,-0.139923,0.202956,1.000000,0.570352,-0.063372,0.014962,-0.091558,0.045091,-0.044811,-0.075545,-0.022415,-0.143078,0.011984,-0.139923,0.202956
statusOS,-0.113228,0.075994,0.570352,1.000000,-0.111111,0.026233,-0.160530,0.079058,-0.078567,-0.132453,-0.028040,-0.045293,-0.083594,-0.113228,0.075994
surgery,0.169842,-0.005687,-0.063372,-0.111111,1.000000,-0.033926,0.171413,0.020329,0.235702,0.132453,-0.118900,-0.001697,0.088748,0.169842,-0.005687
dose,0.117790,-0.199397,0.014962,0.026233,-0.033926,1.000000,-0.132206,0.315258,0.297829,-0.334150,0.468474,0.378734,-0.162929,0.117790,-0.199397
TMZ,0.159428,0.046168,-0.091558,-0.160530,0.171413,-0.132206,1.000000,-0.730626,0.144294,0.515709,0.092544,0.077335,-0.057429,0.159428,0.046168
Chemogroup,-0.153072,0.037592,0.045091,0.079058,0.020329,0.315258,-0.730626,1.000000,0.570208,-0.370692,0.002759,-0.051486,0.013387,-0.153072,0.037592
Chemo,-0.030024,0.110030,-0.044811,-0.078567,0.235702,0.297829,0.144294,0.570208,1.000000,0.083252,0.115345,0.018430,-0.049697,-0.030024,0.110030
diagnostic year,-0.037493,0.089632,-0.075545,-0.132453,0.132453,-0.334150,0.515709,-0.370692,0.083252,1.000000,-0.345445,-0.203701,-0.077207,-0.037493,0.089632


In [23]:
data.drop(columns = ['TMZ', 'gender.1', 'age.1',], axis = 1, inplace= True)

Converting Y to the ndarray so the sksurv package can process it

In [24]:
Y = data[['statusOS', 'OS']]

In [25]:
X = data.drop(columns=['statusOS', 'OS'])

In [26]:
Y['statusOS'] = Y['statusOS'].map({1.0: True, 0: False})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
Y = Y.to_records(index=False)

In [28]:
Y

rec.array([( True, 1.36), ( True, 8.84), ( True, 1.46), ( True, 2.42),
           ( True, 1.26), ( True, 0.85), ( True, 6.43), ( True, 0.63),
           ( True, 0.96), ( True, 1.32), ( True, 2.82), ( True, 0.77),
           ( True, 0.84), (False, 2.38), ( True, 0.23), ( True, 1.  ),
           ( True, 3.5 ), ( True, 1.18), ( True, 5.4 ), ( True, 1.93),
           ( True, 0.59), ( True, 0.48), ( True, 2.25), ( True, 1.34),
           ( True, 0.9 ), ( True, 1.12), ( True, 0.58), ( True, 1.01),
           ( True, 0.88), ( True, 0.62), ( True, 1.59), ( True, 2.33),
           ( True, 1.66), ( True, 1.11), ( True, 0.79), ( True, 1.11),
           ( True, 2.92), ( True, 1.22), ( True, 1.9 ), ( True, 2.5 ),
           ( True, 0.32), ( True, 2.22), ( True, 9.15), ( True, 0.45),
           ( True, 1.37), ( True, 3.45), ( True, 5.07), ( True, 1.82),
           ( True, 0.41), ( True, 0.95), ( True, 0.98), ( True, 1.38),
           ( True, 1.82), ( True, 2.68), ( True, 0.89), ( True, 1.68),
      

# Let's do the ranking

In [ ]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

In [30]:
pip install eli5

     |████████████████████████████████| 106 kB 22.1 MB/s 


In [31]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from eli5.sklearn import PermutationImportance
import eli5


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [32]:
rsf = RandomSurvivalForest()


In [34]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same test 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_rank, y_rank)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much 
    feature_importance = permuter.feature_importances_
    rankings = np.add(feature_importance, rankings)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.Series(ranking_terminal, columns).sort_values(ascending = False)
  c_mean = np.mean(np.array(c_score_total))
  



  return c_rank, c_mean

In [35]:
rank, c_mean = permutation(X, Y, rsf, 100)

In [36]:
rank

PFS                0.201808
age                0.014192
dose               0.007611
diagnostic year    0.005469
Chemogroup         0.002175
Chemo              0.001225
statusPFS          0.000000
surgery           -0.001809
gender            -0.002703
volume            -0.002767
dtype: float64

In [37]:
c_mean

0.7656820876370868

Now let's try the first 2 feature.

In [43]:
X_tf = data[['PFS', 'age']]

In [39]:
def top2features(X, y, model, loops):
  
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
  c_mean_top2 = np.mean(np.array(c_score_total))
  



  return c_mean_top2

In [44]:
c_score_tf = top2features(X_tf, Y, rsf, 100)

In [45]:
c_score_tf 

0.7707494191374952